# Router

[Routing detail documents](https://python.langchain.com/docs/expression_language/how_to/routing)

> Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. <br>
> Routing helps provide structure and consistency around interactions with LLMs.<br>
> `Routing` <font color=blue>允许你创建一个 非确定性 链, 其中上一步的输出定义下一步</font><br>
> `Routing` <font color=blue>提供与LLM交互的结构和一致性</font><br>

> As a very simple example, let's suppose we have two templates optimized for different types of questions, <br>
> and we want to choose the template based on the user input.
> 我们有两个针对<font color=blue>不同类型问题</font>优化的模板.<br>

In [2]:
from langchain.prompts import PromptTemplate

In [10]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

In [11]:
physics_prompt = PromptTemplate.from_template(physics_template)

In [4]:
math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [9]:
math_prompt = PromptTemplate.from_template(math_template)

## LCEL

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

In [7]:
general_prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the question as accurately as you can.\n\n{input}"
)

In [12]:
prompt_branch = RunnableBranch(
    (lambda x: x["topic"] == "math", math_prompt),
    (lambda x: x["topic"] == "physics", physics_prompt),
    general_prompt,
)

In [13]:
from typing import Literal

from langchain.pydantic_v1 import BaseModel
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [14]:
class TopicClassifier(BaseModel):
    "Classify the topic of the user question"

    topic: Literal["math", "physics", "general"]
    "The topic of the user question. One of 'math', 'phsyics' or 'general'."

In [15]:
classifier_function = convert_pydantic_to_openai_function(TopicClassifier)

In [16]:
llm = ChatOpenAI().bind(
    functions=[classifier_function], function_call={"name": "TopicClassifier"}
)

In [17]:
parser = PydanticAttrOutputFunctionsParser(
    pydantic_schema=TopicClassifier, attr_name="topic"
)

### 先用OpenAI 做分类

In [18]:
classifier_chain = llm | parser

In [19]:
from operator import itemgetter

from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [20]:
final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
    | prompt_branch
    | ChatOpenAI()
    | StrOutputParser()
)

In [21]:
final_chain.invoke(
    {
        "input": "What is the first prime number greater than 40 such that one plus the prime number is divisible by 3?"
    }
)

"Thank you for your kind words! I'm here to help you with your math question.\n\nTo find the first prime number greater than 40, we'll start by listing out prime numbers in ascending order until we find the desired prime number that satisfies the given condition.\n\nPrime numbers greater than 40: 41, 43, 47, 53, 59, 61, 67, ...\n\nNow, we need to check if one plus each of these prime numbers is divisible by 3. We can do this by calculating the remainder when dividing by 3.\n\n(41 + 1) % 3 = 42 % 3 = 0 (divisible by 3)\n(43 + 1) % 3 = 44 % 3 = 2 (not divisible by 3)\n(47 + 1) % 3 = 48 % 3 = 0 (divisible by 3)\n(53 + 1) % 3 = 54 % 3 = 0 (divisible by 3)\n(59 + 1) % 3 = 60 % 3 = 0 (divisible by 3)\n(61 + 1) % 3 = 62 % 3 = 2 (not divisible by 3)\n(67 + 1) % 3 = 68 % 3 = 2 (not divisible by 3)\n...\n\nFrom the above calculations, we can see that the first prime number greater than 40, where one plus the prime number is divisible by 3, is 41. Therefore, the answer is 41.\n\nHence, the first 

### 自定义路由器

```python
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain
```